In [120]:
import matplotlib.pyplot as plt
%matplotlib inline

In [121]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
#! wget -o  ../../datasets/ https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz
column_names = 'label, lepton pT, lepton eta, lepton phi, missing energy magnitude, missing energy phi, jet 1 pt, jet 1 eta, jet 1 phi, jet 1 b-tag, jet 2 pt, jet 2 eta, jet 2 phi, jet 2 b-tag, jet 3 pt, jet 3 eta, jet 3 phi, jet 3 b-tag, jet 4 pt, jet 4 eta, jet 4 phi, jet 4 b-tag, m_jj, m_jjj, m_lv, m_jlv, m_bb, m_wbb, m_wwbb'.split(', ')
data = pd.read_csv('../../datasets/HIGGS.csv.gz', header=None, names=column_names)

In [122]:
X_data = data.drop('label', 1)
X_data = StandardScaler().fit_transform(X_data)

labels = data["label"]

In [123]:
from sklearn.model_selection import train_test_split
import numpy as np
indx_train, indx_test = train_test_split(np.arange(len(labels), dtype='int32'), stratify=labels, train_size=1e-2, test_size=0.1)

In [124]:
y_train = labels[indx_train]
y_test = labels[indx_test]

X_train = X_data[indx_train]
X_test = X_data[indx_test]

In [125]:
X_good = X_train[np.where(y_train==0)[0]]
X_bad = X_train[np.where(y_train==1)[0]]

In [126]:
X_good.shape, X_bad.shape, X_train.shape, X_data.shape

((51709, 28), (58291, 28), (110000, 28), (11000000, 28))

In [141]:
import theano
import theano.tensor as T
import lasagne
from lasagne.layers import InputLayer, DenseLayer
from lasagne.nonlinearities import softmax
from lasagne.objectives import categorical_crossentropy

input_size = X_good.shape[1] # features cnt

input_var = T.matrix('input', dtype='float32')
weights_var = T.vector('weights', dtype='float32')
target_var = T.matrix('target', dtype='float32')
lr_var = T.scalar('learning rate')

network = InputLayer(shape=(None, input_size), input_var=input_var)
network = DenseLayer(network, 400)
network = DenseLayer(network, 2, nonlinearity=softmax)
output = lasagne.layers.get_output(network)

loss = (weights_var * categorical_crossentropy(output, target_var)).mean()
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.rmsprop(loss, params, learning_rate=lr_var)

train_fn = theano.function([input_var, weights_var, target_var, lr_var], [output, loss], updates=updates, allow_input_downcast=True)
predict_fn = theano.function([input_var], [output], allow_input_downcast=True)

In [134]:
X_min, X_max = np.min(X_data, axis=0), np.max(X_data, axis=0)

def random_sampling(size, features_range=[X_min, X_max]):
    return np.array([features_range[0][i] * np.ones(size) + np.random.rand(size) * (features_range[1][i] - features_range[0][i]) for i in range(len(features_range[0]))]).transpose()

In [135]:
import sys
sys.path.append('../../')
from hmc import *

In [144]:
# from sklearn.metrics import *
from evaluation import get_anomaly_metrics
from keras.utils import to_categorical

lr = 1e-4
epoches = 5000
batch_size = 10000
sampler_batch_size = 10000
sampler_epoch_rate = 1
fraction = 10
alpha = 1e-3

true_bad_size = int(alpha * fraction * y_train.sum())
fake_bad_size = int((1.0 - alpha) * fraction * y_train.sum())
good_size = X_good.shape[0]

X_bad_true = X_train[y_train == 1][np.random.choice(np.arange(int(y_train.sum())), size=true_bad_size), :]
X_bad_fake = random_sampling(sampler_batch_size)


target = np.concatenate([np.zeros(good_size), np.ones(true_bad_size + fake_bad_size)])
good_weights = np.ones(X_good.shape[0])
true_bad_weights = alpha * np.ones(true_bad_size)
fake_bad_weights = (1 - alpha) * np.ones(fake_bad_size)

position = theano.shared(X_bad_fake)

def P(x):
    preds = x
    for layer in lasagne.layers.get_all_layers(network)[1:]: # pass x variable through all network layers except input one
        preds = layer.get_output_for(preds)
    preds = preds[:, 1]
    return 1 - T.exp(-preds / (1 - preds))

def get_P(x):
    preds = predict_fn(x)[0].squeeze()[:, 1]
    return 1 - np.exp(-preds / (1 - preds))

sampler = HMC_sampler.new_from_shared_positions(position, P,
                      initial_stepsize=1e-3, stepsize_max=0.5)

In [146]:
true_bad_size, fake_bad_size, good_size

(582, 582327, 51709)

In [147]:
aucs = [0.4, 0.41]
for epoch in range(epoches):
    if epoch % 150 == 0 and epoch > 0: # and aucs[-1] - np.mean(aucs[-6:-1]) < 1e-4:
        lr /= 3
    print('Epoch: %d' % epoch)
    
    [sampler.draw() for _ in range(sampler_epoch_rate)]
    
    X_bad_fake = np.concatenate([sampler.draw() for _ in range(int(round(fake_bad_size / sampler_batch_size)))][:fake_bad_size])
    X_mix = np.concatenate([X_good, X_bad_fake, X_bad_true])
    
    eps = 1e-4
    fake_bad_weights = 1./(eps + get_P(X_bad_fake))
#     fake_bad_weights /= max(fake_bad_weights)
    fake_bad_weights *= (1. - alpha)
    
    
    weights = np.concatenate([good_weights, fake_bad_weights, true_bad_weights])
    indices = np.arange(len(weights))
    np.random.shuffle(indices)
    for i in range(int(len(indices)/batch_size)):
        batch_idx = indices[range(batch_size*i,min(batch_size*(i+1), len(indices)))]
        _, loss_value = train_fn(X_mix[batch_idx], weights[batch_idx], to_categorical(list(target[batch_idx]) + [0,1])[:-2], lr)
    y_pred = predict_fn(X_test)[0].squeeze()[:, 1]
    metrics = get_anomaly_metrics(y_test, y_pred)
    aucs.append(metrics['PR_AUC'])
    print(metrics)
    
    print('Learning rate: ', lr)

Epoch: 0
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5971278240124236, 'Precision@0.95': 0.529919572654157, 'P@10': 0.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.5728007231626207}
('Learning rate: ', 0.0001)
Epoch: 1
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5972689301664321, 'Precision@0.95': 0.529919572654157, 'P@10': 0.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.5729033911890004}
('Learning rate: ', 0.0001)
Epoch: 2
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5975888295160403, 'Precision@0.95': 0.529919572654157, 'P@10': 0.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.5731202536579952}
('Learning rate: ', 0.0001)
Epoch: 3
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5976707949602994, 'Precision@0.95': 0.529919572654157, 'P@10': 0.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.57315735247428

{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.6003936010972903, 'Precision@0.95': 0.529919572654157, 'P@10': 0.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.5748375275593184}
('Learning rate: ', 0.0001)
Epoch: 33
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.6004076456597653, 'Precision@0.95': 0.529919572654157, 'P@10': 0.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.5748101480420089}
('Learning rate: ', 0.0001)
Epoch: 34
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.6004601453152105, 'Precision@0.95': 0.529919572654157, 'P@10': 0.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.5748437512070992}
('Learning rate: ', 0.0001)
Epoch: 35
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.6005400243801889, 'Precision@0.95': 0.529919572654157, 'P@10': 0.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.574888891108022}
('L

{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.6013298923272157, 'Precision@0.95': 0.529919572654157, 'P@10': 0.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.57509187736973}
('Learning rate: ', 0.0001)
Epoch: 65
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.6013748299644104, 'Precision@0.95': 0.529919572654157, 'P@10': 0.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.5751173121314735}
('Learning rate: ', 0.0001)
Epoch: 66
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.6013915829075567, 'Precision@0.95': 0.529919572654157, 'P@10': 0.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.5751153645658454}
('Learning rate: ', 0.0001)
Epoch: 67
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.601430099448087, 'Precision@0.95': 0.529919572654157, 'P@10': 0.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.5751398367269648}
('Lea

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
import evaluation
reload(evaluation)
from evaluation import get_anomaly_metrics

In [ ]:
Score = y_pred

precision, recall, _ = precision_recall_curve(y_test, Score)
pr_auc = auc(recall, precision)

plt.figure()
plt.title('(1 + e) with Hamilton sampling')
plt.plot(recall, precision, label='PR curve (area = %0.3f)' % pr_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

fpr, tpr, _ = roc_curve(y_test, Score)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.title('(1 + e) with Hamilton sampling')
plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()